In [1]:
import numpy as np
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2

#import opencv as cv2

img_array = np.load('test_images.npy',encoding='latin1')



In [2]:
images = np.load('train_images.npy',encoding='latin1')
len(images)


10000

In [3]:
input=[]
for i in range(len(images)):
    image1 = (images[i][1]).reshape(100,100)
    #plt.imshow(image1)
    img = cv2.imwrite('temp.jpg',image1)
    img = cv2.imread('temp.jpg',0)
    edges = cv2.Canny(img, 0, 100)
    #plt.imshow(edges)

    im2, contours, hierarchy = cv2.findContours(edges, cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:3]


    mask=np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, cnts, -1, (255),1)
    input.append(mask)
    #plt.imshow(mask)


In [4]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

labels = np.loadtxt('train_labels.csv',dtype=str, encoding='latin1',delimiter=',')
X = [x.reshape(10000,) for x in input]
y = labels[1:,1]


svm = LinearSVC(max_iter=50000)
svm_params = {
    "tol": uniform(0,1e-2),
    "C": uniform(0,10),
    "dual": (True, False)
}

baseline = RandomizedSearchCV(svm, svm_params, n_iter=2000,verbose=1)


baseline.fit(X,y)
print("done???")

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2000 candidates, totalling 6000 fits


[Parallel(n_jobs=1)]: Done 6000 out of 6000 | elapsed: 254.3min finished


done???


In [5]:
# produce prediction file
result = np.loadtxt('sample_submission.csv',dtype=str, encoding='latin1',delimiter=',')
test_array = img_array[:,1].tolist()
prediction = baseline.predict(test_array)
print(prediction)
for i in range(len(prediction)):
    result[i+1][1]=prediction[i]

print(result)
    
np.savetxt('svm_submission.csv', result, delimiter=',', fmt="%s")

['moustache' 'octagon' 'empty' ... 'nail' 'sink' 'nail']
[['Id' 'Category']
 ['0' 'moustache']
 ['1' 'octagon']
 ...
 ['9997' 'nail']
 ['9998' 'sink']
 ['9999' 'nail']]
